In [1]:
import base64
import logging
import mimetypes
import os
import os.path
import pickle
import uuid

import cgi,html
import uuid
from email.mime.multipart import MIMEMultipart
from email.mime.text      import MIMEText
from email.mime.image     import MIMEImage
from email.header         import Header


from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient import errors
from googleapiclient.discovery import build

In [9]:
def get_service(pathToCredentials,dirToPickle):
    """Gets an authorized Gmail API service instance.
    
    pathToCredentials: path, including name, of the credentials .json file

    Returns:
        An authorized Gmail API service instance..
    """    

    # If modifying these scopes, delete the file token.pickle.
    SCOPES = [
        'https://www.googleapis.com/auth/gmail.readonly',
        'https://www.googleapis.com/auth/gmail.send',
    ]

    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    picklePath = os.path.join(dirToPickle,'token.pickle')
    if os.path.exists(picklePath):
        with open(picklePath, 'rb') as token:
            creds = pickle.load(token)
            
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                pathToCredentials, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(picklePath, 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service
    
def send_message(service, sender, message):
    """Send an email message.

    Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

    Returns:
    Sent Message.
    """
    try:
        sent_message = (service.users().messages().send(userId=sender, body=message).execute())
        logging.info('Message Id: %s', sent_message['id'])
        return sent_message
    except errors.HttpError as error:
        logging.error('An HTTP error occurred: %s', error)
    
def create_message(sender, to_list, subject, message_text,img1):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    
    message = MIMEMultipart('related')

    msg_content = '''
    <html><body><p>%s</p><p><img src="cid:screenshot" width="250" height="250"></p></body></html>'''%(message_text)
    message.attach(MIMEText((msg_content), 'html'))

    with open(img1["path"], 'rb') as image_file:
        image = MIMEImage(image_file.read())
    image.add_header('Content-ID', '<screenshot>')
    image.add_header('Content-Disposition', 'inline', filename=img1["path"])
    message.attach(image)

    
    message['From'] = sender
    message['To'] = ','.join(to_list)
    message['Subject'] = subject    
    
    s = message.as_string()
    b = base64.urlsafe_b64encode(s.encode('utf-8'))
    return {'raw': b.decode('utf-8')}

    
if __name__ == '__main__':
    
    # used info from these websites in particular to generate this code:
    # http://dogdogfish.com/python-2/emailing-multiple-inline-images-in-python/
    # https://stackoverflow.com/questions/26630069/python-smtplib-and-mimetext-adding-an-attachment-to-an-email
    # https://stackoverflow.com/questions/25944883/how-to-send-an-email-through-gmail-without-enabling-insecure-access
    
    logging.basicConfig(
        format="[%(levelname)s] %(message)s",
        level=logging.INFO
    )

    try:
        # static parameters to hard code
        fromEmail = "stl.electroplating@gmail.com"
        toEmail = ["pound.ben@gmail.com","bpound@ucla.edu"]
        pathToScreenshot = "C:\\Users\\Ben\\Downloads"
        nameScreenshot = 'smallfile.png'
        pathToCredentials = "C:\\Users\\Ben\\Desktop\\credentials.json"
        dirToPickle="C:\\Users\\Ben\\Desktop"
        
        # dynamic quantities to calculate
        subject = 'new subject'
        body = 'new body'
        img1 = dict(title = 'desktop screenshot', path = os.path.join(pathToScreenshot,nameScreenshot) , cid = str(uuid.uuid4()))
        
        service = get_service(pathToCredentials,dirToPickle)
        message = create_message(fromEmail,toEmail, "Test subject", "Test body",img1)
        send_message(service, fromEmail, message)

    except Exception as e:
        logging.error(e)
        raise
    """
    
    img1 = dict(title = 'desktop screenshot', path = '009.jpg', cid = str(uuid.uuid4()))
    # img2 = dict(title = 'Image 2', path = 'test_image_2.png', cid = str(uuid.uuid4()))

    email_msg = generate_email(fromEmail, ['recipient@email.com'], 'test_data.txt', 'test_data_2.txt', img1, img2)
    send_email(email_msg, fromEmail, gmail_pwd, ['recipient@email.com'])
    """

[INFO] file_cache is only supported with oauth2client<4.0.0
[INFO] Message Id: 1783d7d15ed68bdf
